In [8]:
from bs4 import BeautifulSoup
import numpy as np
from csv import writer
import glob
from os.path import exists

interval = 0.142857143
emotions_list = ['joy', 'surprise', 'anticipation', 'nostalgia', 'sorrow', 'boredom', 'annoyance', 'rage']

directory='saves'   #this is the directory with the xml files
for filename in glob.iglob(f'{directory}/*.xml'):
	
	# Reading the data inside the xml file to a variable under the name data
	with open(filename, 'r') as f:
		data = f.read()
		# Passing the stored data inside the beautifulsoup parser, storing the returned object
		Bs_data = BeautifulSoup(data, "xml")
		
		# Find the instance of 'exitText' to get the region of the songs in the last word (or two last words)
		reg = Bs_data.find('exitText')
		region = reg.text.split()
		if (reg.text.endswith('Songs.') or reg.text.endswith('Songs') or reg.text.endswith('Music')):
			region = region[-2]
		#	print(region)
		else:
			region = region[-1]
		
		if (reg.text.endswith('Greece/Evoia')):
			region = 'CentralGreece_Evoia'	
		
		if (region=='Minor'):
			region = 'Mikra_Asia'
		if (region=='Gulf'):
			region = 'Saronic_Gulf'
		if (region=='Islands'):
			region = 'Ionian_Islands'
		
		tmp = reg.text.split()
		if (region=='Aegean' and tmp[-3]=='North'):
			region = 'North_Aegean'
		if (region=='Aegean' and tmp[-3]=='East'):
			region = 'East_Aegean'
			
			
		
		
		# Finding all instances of tag 'value' that has the votes of the songs
		b_unique = Bs_data.find_all('value')
		total_songs = len(b_unique)
		
		print('Now processing file:',filename, 'from region ',region, 'with ', total_songs, 'songs')
		if (total_songs==0):
			print('No responses from subject found! - Skipping this file (no output will be written)')
			continue

		# construct the list with the emotion that was assigned to each song
		emotions=list()
		for count, tags in enumerate(b_unique):
			res = float(tags.text.strip())
			emo = emotions_list[int(round(res/interval))]
			emotions.append(emo)

		
		# this is to make the survey array
		surv = Bs_data.find_all("response")
	
		#make an array with all the responses
		surv_array=np.zeros(35, dtype=int)
	
		for count2, tags in enumerate(surv):
			resp = tags.get("checked")
        	#the results show the true responses in the survey
			if (resp == "true" and count2<35):
				name = tags.get("name")
				surv_array[count2]=1
        	
		for count2, tags in enumerate(surv):
			resp = tags.get("checked")
    		#the results show the extra responses in 3 questions
			if (resp != "true" and resp !="false" and count2<35):
				name = tags.get("name")
        
		for count2, tags in enumerate(surv):
			if (count2==2):
				age=int(tags.text)
				surv_array[2]=int(age)
		
		#print(surv_array)
		
		
		array_to_write = np.concatenate((emotions,surv_array), axis=0)
		#this one makes a csv file with all grades inside. It must find out the region that the xml belongs to, and then open for append the appropriate csv file.
		# I suggest to give filenames the same with the regions for simplicity and for easier processing. 
		
		#need to make a header for the csv
		headerlist = list(range(1,count+2))+ ['Male', 'Female', 'Age','Crete','Cyprus','Peloponnese','Central Greece','Thessaly', 'Epirus','Macedonia','Thrace','Pontos','Dodekanisa','Cyclades','Sporades','Ionnian','North_Agean','Other','Place_extra','Elementary','High School', 'Senior School','IEK','AEI-TEI', 'Post-grad_PhD','Music relation (Y)','Music relation (N)','Music rel (extra)','Classic','Jazz/Blues','Pop','Rock_Metal','Electronical','Greek Music','Rap_Hip-Hop']		
		
		fname = region + '.csv'
		file_exists = exists(fname)
		with open(fname, 'a', newline='') as f1:
			writer_object = writer(f1)
			if (file_exists==False):
				writer_object.writerow(headerlist)
			writer_object.writerow(array_to_write)
			f1.close()



Now processing file: saves/save-bI519PTJoSR62jCLL5vNUHGpfRL4nKDz.xml from region  Athenian with  11 songs


AttributeError: 'list' object attribute 'append' is read-only